<hr style='color:#BA704F;'>
<p style='color:#6C3428;font-size:20px;text-align:center;'>GeoLocation</p>
<hr style='color:#BA704F;'>

In [7]:
import geopandas as Gpd
import geodatasets as Gds
import matplotlib.pyplot as Mpl
import contextily as cx
import folium
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy.distance import geodesic as GD

In [82]:
#location of your file
File_Path = 'C:\\GeoLocation\\map.geojson'
File_Path1 = 'C:\\GeoLocation\\points.csv'

In [83]:
# dataframe
df1 = Gpd.read_file(File_Path1)

In [93]:
m = folium.Map(location=[32.246844749437734,-8.52468086856831], zoom_start=17)

# for index,x in df1.iterrows():
#     # folium.Marker(
    #     location=[x.lat,x.lon],
    #     popup=x.Name,
    #     icon=folium.Icon(icon="star"),
    # ).add_to(m)
Line =[[
          -8.521798483107432,
          32.24752699164587
        ],
        [
          -8.521951724885213,
          32.248675641029536
        ]
        ]
folium.Marker(location=[
          32.248675641029536,-8.521951724885213
        ],popup='hello',
        icon=folium.Icon(color='red')).add_to(m)
m

In [72]:
tile = folium.TileLayer(
      tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
      attr = 'Esri',
      name = 'Esri Satellite',
      overlay = True,
      control = True
      ).add_to(m)

m = folium.Map(location=[32.246844749437734, -8.52468086856831], zoom_start=16, tiles=tile)

for _, r in df2.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = Gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "red"})
    folium.Popup(r['Name']).add_to(geo_j)
    geo_j.add_to(m)

m


<hr style='color:#BA704F;'>
<p style='color:#6C3428;font-size:20px;text-align:center;'>Geocoders</p>
<hr style='color:#BA704F;'>

In [193]:
df2 = Gpd.read_file(File_Path)
df2

,Name,geometry
0,Line To Youcode,"LINESTRING (-8.52156 32.24710, -8.52163 32.247..."


In [228]:
def getCityOnMapByName(Name,User_Agent):
    global m
    df = pd.DataFrame({'name': [Name]})
    geolocator = Nominatim(user_agent=User_Agent)

    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
    df['location'] = df['name'].apply(geocode)

    df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

    tile = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True,
        detect_retina=True
        )

    m = folium.Map(location=[df['point'][0][0], df['point'][0][1]], zoom_start=16, tiles=tile)

    for _, r in df2.iterrows():
        # Without simplifying the representation of each borough,
        # the map might not be displayed
        sim_geo = Gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001,)
        geo_j = sim_geo.to_json()
        geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "red"})
        popup_text = f'{r["Name"]}<br>{geo_j.get_bounds()}'
        folium.Popup(popup_text).add_to(geo_j)
        geo_j.add_to(m)

    return m

In [229]:
getCityOnMapByName('Youssoufia','ecode')

In [120]:
def getdistanceOnMap(City_1,City_2,User_Agent):
    df = pd.DataFrame({'name': [City_1,City_2]})
    geolocator = Nominatim(user_agent=User_Agent)

    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
    df['location'] = df['name'].apply(geocode)

    df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
    safi = (df['point'][0][0],df['point'][0][1])
    Youssoufia = (df['point'][1][0],df['point'][1][1])
    print("The distance between Youssoufia and Safi is :",GD(df['point'][0],df['point'][1]).km)
    print(safi)
    print(Youssoufia)
getdistanceOnMap('Youssoufia','Safi','ecode')

The distance between Youssoufia and Safi is : 65.81814535663293
(32.245801, -8.532439)
(32.2650776, -9.230554032530662)


In [101]:
df2

,Name,geometry
0,Youcde,POINT (-8.52180 32.24753)
1,Hopital Cheikh Zaid,POINT (-8.52195 32.24868)
2,NaN,"LINESTRING (-8.52156 32.24712, -8.52164 32.247..."
3,NaN,"POLYGON ((-8.52504 32.24733, -8.52545 32.24567..."
